In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [4]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [5]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [6]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [20]:
list_words = ['dancer', 'woman', 'data', 'english']

In [21]:
for word in list_words:
    print(f'The similar words to {word} are:\n{glove_wiki_vectors.most_similar(word)}')

The similar words to dancer are:
[('choreographer', 0.7776089310646057), ('ballerina', 0.7597374320030212), ('performer', 0.7322809100151062), ('dancers', 0.718708336353302), ('ballet', 0.7059119939804077), ('actress', 0.6762431263923645), ('dance', 0.6759555339813232), ('singer', 0.6707386374473572), ('musician', 0.6590761542320251), ('pianist', 0.6540020108222961)]
The similar words to woman are:
[('girl', 0.8472672700881958), ('man', 0.8323494791984558), ('mother', 0.827568769454956), ('boy', 0.7720510363578796), ('she', 0.7632068395614624), ('child', 0.7601761817932129), ('wife', 0.7505022883415222), ('her', 0.7445706129074097), ('herself', 0.7426273822784424), ('daughter', 0.7264456152915955)]
The similar words to data are:
[('information', 0.7920401096343994), ('analysis', 0.7564454674720764), ('tracking', 0.7226751446723938), ('database', 0.7215273976325989), ('system', 0.6736716032028198), ('computer', 0.6733997464179993), ('statistics', 0.6705881953239441), ('systems', 0.66958

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [22]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [27]:
for pairs in word_pairs:
    print(f'The cosine similarity between {pairs[0]} and {pairs[1]} is: {glove_wiki_vectors.similarity(pairs[0], pairs[1])}')

The cosine similarity between coast and shore is: 0.7000272274017334
The cosine similarity between clothes and closet is: 0.5462759733200073
The cosine similarity between old and new is: 0.6432487964630127
The cosine similarity between smart and intelligent is: 0.7552732229232788
The cosine similarity between dog and cat is: 0.8798074722290039
The cosine similarity between tree and lawyer is: 0.07671945542097092


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [26]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [31]:
def in_wiki_embeds(word):
    '''
    returns True if given word is in Wikipedia embeddings vocabulary, False if not
    '''
    return word in glove_wiki_vectors

In [32]:
for word in test_words:
    if in_wiki_embeds(word):
        print(f'The word {word} is in the Wikipedia embeddings.')
    else:
        print(f'The word {word} is not in the Wikipedia embeddings.')

The word covididiot is not in the Wikipedia embeddings.
The word fomo is not in the Wikipedia embeddings.
The word frenemies is in the Wikipedia embeddings.
The word anthropause is not in the Wikipedia embeddings.
The word photobomb is not in the Wikipedia embeddings.
The word selfie is not in the Wikipedia embeddings.
The word pxg is not in the Wikipedia embeddings.
The word pacg is not in the Wikipedia embeddings.
The word cct is in the Wikipedia embeddings.
The word escc is in the Wikipedia embeddings.


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [33]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [34]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [35]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283238

In [36]:
glove_wiki_vectors.similarity("white", "success")

0.35182396

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [56]:
analogy("woman", "caretaker", "man")

woman : caretaker :: man : ?


,Analogy word,Score
0,interim,0.582048
1,boss,0.574412
2,manager,0.534280
3,appointed,0.528382
4,prime,0.523247
5,premier,0.519685
6,sacked,0.518245
7,pleat,0.507855
8,venables,0.501967
9,successor,0.499100


In [76]:
analogy("scientist", "man", "homemaker")

scientist : man :: homemaker : ?


,Analogy word,Score
0,housewife,0.690942
1,mother,0.589116
2,mom,0.583259
3,woman,0.581701
4,girl,0.578157
5,grandmother,0.568886
6,schoolteacher,0.566959
7,waitress,0.553455
8,shopkeeper,0.547613
9,girlfriend,0.545629


In [83]:
glove_wiki_vectors.similarity("blonde", "smart")

0.3121177

In [84]:
glove_wiki_vectors.similarity("blonde", "dumb")

0.4982603

In [89]:
analogy("server", "boss", "woman")

server : boss :: woman : ?


,Analogy word,Score
0,man,0.730424
1,girl,0.640664
2,father,0.635410
3,boy,0.634364
4,husband,0.616292
5,wife,0.615824
6,brother,0.611986
7,friend,0.610112
8,boyfriend,0.602780
9,son,0.600712


In [93]:
analogy("boy", "strong", "girl") # a de-biased example

boy : strong :: girl : ?


,Analogy word,Score
0,stronger,0.757472
1,despite,0.698031
2,strongest,0.697400
3,weak,0.680318
4,solid,0.677007
5,very,0.676151
6,robust,0.671619
7,particularly,0.667278
8,consistent,0.662579
9,good,0.657380


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

It seems that gender bias is still present in the Wikipedia embeddings, depsite some trials that appeared to be de-biased. Classical stereotypes such as 'dumb blondes' also seem to peek through in these pre-trained embeddings. Biased embeddings like these can be incredibly harmful, as they perpetuate stereotypes, regardless of whether or not these stereotypes are valid or accurately reflect society. In most cases, stereotypes like these stand to harm more than they help. Biased embeddings in particular can be incredibly harmful if, for example, a news website were to use topic modelling to cluster articles based on their contents and clustered articles based solely on stereotypes rather than objective categories. This could be highly offensive to certain groups. It is important to be aware of these biases and recognize their limitations on a model.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [94]:
from sklearn.datasets import fetch_20newsgroups

In [95]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [96]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

In [120]:
test_df = df[:50]
test_df.head()

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. 922 sence wonder help u.s.c. ...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [98]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [132]:
# following code adapted from Lecture 18 notes

def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE", "NUM", "EMAIL", "URL"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [129]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]

In [133]:
test_df["text_pp"] = [preprocess(text) for text in nlp.pipe(test_df["text"])]

/var/folders/gx/0f3c0wm11cs4n_87_vsk0fhw0000gn/T/ipykernel_9541/2898573282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["text_pp"] = [preprocess(text) for text in nlp.pipe(test_df["text"])]


In [134]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

Because we know the true number of topics here is 8, I decided to choose a range including 8. However, I believe the topics that were included in the subset of topics in our sample can be reduced to 4: sports, religion, technology, and politics. As such, I decided to pick a range from 4-8 increasing by one using ``numpy's arange``. I believe this range will capture at least one meaningful topic segmentation. Below I perform topic modelling 5 times using ``LatentDirichletAllocation`` and ``mglearn``.

In [160]:
# following code adapted from Lecture 18

from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(stop_words='english', strip_accents='ascii')
X = vec.fit_transform(df["text_pp"])

In [161]:
# following code adapted from Lecture 18

from sklearn.decomposition import LatentDirichletAllocation
import mglearn


def model_topics(n_topics: int, X=X):
    '''
    takes a number of topics and performs topic modelling on test dataset containing preprocessed newsgroup text with that number of topics
    '''
    lda = LatentDirichletAllocation(n_components = n_topics,
                                    learning_method="batch", 
                                    max_iter=10, 
                                    random_state=42,
                                    n_jobs=-1)

    # fit topic modelling transformer to pre-processed count vectorized text data
    news_topics = lda.fit_transform(X)

    sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
    feature_names = np.array(vec.get_feature_names_out()) 
    
    mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=10,
    n_words=10)

In [162]:
topics = np.arange(4, 9, 1)

print(topics)

for topic in topics:
    model_topics(topic)

[4 5 6 7 8]
topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
people        file          god           game          
say           program       people        team          
know          use           think         year          
gun           window        know          play          
armenian      edu           believe       good          
time          server        jesus         player        
come          image         say           win           
kill          entry         thing         season        
think         widget        question      think         
armenians     display       time          hockey        


topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
people        god           god           game          file          
say           jesus         believe       team          image         
know          isra

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [163]:
lda = LatentDirichletAllocation(n_components = 4,
                                    learning_method="batch", 
                                    max_iter=10, 
                                    random_state=42,
                                    n_jobs=-1)

# fit topic modelling transformer to pre-processed count vectorized text data
final_news_topics = lda.fit_transform(X)

sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out()) 
    
mglearn.tools.print_topics(
    topics=range(4),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=10,
    n_words=10)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
people        file          god           game          
say           program       people        team          
know          use           think         year          
gun           window        know          play          
armenian      edu           believe       good          
time          server        jesus         player        
come          image         say           win           
kill          entry         thing         season        
think         widget        question      think         
armenians     display       time          hockey        




#### Interpretation of clustering with suggested labels

Above, I believe I have captured the segmentation I aimed to capture via my preprocessing steps. Here, topic 0 represents politics, topic 1: technology, topic 2: religion, and topic 3: sports. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [166]:
assignments = {'document': [],
               'politics': [],
               'technology': [],
               'religion': [],
               'sports': []}

list_topics = ['politics', 'technology', 'religion', 'sports']

for i in range(5):
    assignments['document'].append(df.iloc[i]['text'])

    for n in range(4):
        assignments[list_topics[n]].append(final_news_topics[i][n])

pd.DataFrame(assignments)

,document,politics,technology,religion,sports
0,"You know, I was reading 18 U.S.C. 922 and some...",0.284275,0.005058,0.661248,0.049420
1,\n\n\nIt's not a bad question: I don't have an...,0.399606,0.137476,0.459120,0.003799
2,"\nActuallay I don't, but on the other hand I d...",0.244973,0.204181,0.546133,0.004713
3,"The following problem is really bugging me,\na...",0.009497,0.971745,0.009950,0.008807
4,\n\n This is the latest from UPI \n\n For...,0.981922,0.005952,0.006180,0.005945


In [168]:
for i in range(5):
    print(f"Document {i}:\n{df.iloc[i]['text']}\n")

Document 0:
You know, I was reading 18 U.S.C. 922 and something just did not make 
sence and I was wondering if someone could help me out.

Say U.S.C. 922 :

(1) Except as provided in paragraph (2), it shall be unlawful for
any person to transfer or possess a machinegun.

 Well I got to looking in my law dictionary and I found that a "person" 
might also be an artificial entity that is created by government 
and has no rights under the federal constitution. So, what I 
don't understand is how a statute like 922 can be enforced on 
an individual. So someone tell me how my government can tell
me what I can or cannot possess. Just passing a law 
does not make it LAW. Everyone knows that laws are constitional
until it goes to court. So, has it ever gone to court, not
just your run of the mill "Ok I had it I am guilty, put me in jail"

Has anyone ever claimed that they had a right to possess and was told
by the Supreme Court that they didn't have that right?




Document 1:



It's not a ba

#### Document topic assignments commentary

The first document (about gun laws and the right to possess) seems to have been classified as an intersection between politics and religion, but is arguably more about politics than it is religion. The second document, a mathematical algorithm, seems like it belongs in the technology classification, but has been horribly misclassified in politics and religion again. I am uncertain about the contents of document three, but I can tell it belongs in the technology category while it was once again misclassified. The fourth document, clearly about technology, was rightfully classified as so with 97% of its contents pertaining to the 'technology' cluster. The final document, clearly about war and politics, was rightfully classified with 98% of its contents pertaining to the 'politics' category.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)